# JulES as a long-term analysis model

### Import packages

In [ ]:
using Pkg; Pkg.status()
# Pkg.add("Revise"); Pkg.add("Plots"); Pkg.add("PlotlyJS"); Pkg.add("PrettyTables")
# Pkg.update("TuLiPa") # uncomment to update TuLiPa to latest version
# Pkg.develop(path=joinpath(dirname(dirname(pwd())),"TuLiPa")); Pkg.status() # go to development version
# Pkg.undo(); Pkg.status() # go back to main package version
# Pkg.add(url="https://github.com/NVE/TuLiPa.git"); Pkg.status() # alternative go back to latest version

In [ ]:
using DataFrames, Statistics, JSON, Distributed, Clustering, YAML, CSV, Plots, PrettyTables, Revise
# plotlyjs(); # uncomment for interactive plots

In [ ]:
# config = YAML.load_file(joinpath("data", "config_jules_la.yml")) # config without datasets
config = YAML.load_file(joinpath(dirname(dirname(pwd())), "JulESIO", "config_jules_la_demo_stoch.yml")) # config with NVE datasets
weatheryear = config["main"]["weatheryears"][1]
datayear = config["main"]["datayears"][1];

### Prepare parallell processing - import code on all cores

In [ ]:
const numcores = config["main"]["numcores"]

if nprocs() < numcores
    addprocs(numcores - nprocs())
end

@show nprocs();

In [ ]:
@everywhere using TuLiPa, Dates
# @everywhere include(joinpath(dirname(dirname(dirname(pwd()))),"jgrc/TuLiPa/src/TuLiPa.jl"));

In [ ]:
@everywhere using JulES

### Read dataset

In [ ]:
function getdataset(config, scenarioyear)
    iprogtype = get(config["main"], "iprogtype", "direct")
    useifm = iprogtype != "direct"

    settings = config[config["main"]["settings"]]

    sti_thema = joinpath(config["main"]["inputpath"], "datasett", "data_fra_thema")
    sti_vannkraft = joinpath(config["main"]["inputpath"], "datasett", "data_fra_dynmodell")

    tsd = JSON.parsefile(joinpath(sti_thema, "dataset_thema.json"))
    themastructure = TuLiPa.getelements(tsd, sti_thema)
    tst = JSON.parsefile(joinpath(sti_thema, "dataset_thema_excl_hydro_nose.json"))
    themastructure_exl = TuLiPa.getelements(tst, sti_thema)
    tse = JSON.parsefile(joinpath(sti_thema, "tidsserier_thema.json"))
    themaseries = TuLiPa.getelements(tse, sti_thema)

    dse = JSON.parsefile(joinpath(sti_vannkraft, "tidsserier_detd.json"))
    detdseries = TuLiPa.getelements(dse)
    dst = JSON.parsefile(joinpath(sti_vannkraft, "dataset_detd.json"))
    detdstructure = TuLiPa.getelements(dst)

    progelements = vcat(themaseries, themastructure)

    ifm_replacemap = Dict{String, String}()
    ifm_names = String[]
    ifm_weights = Dict{String, Dict{String, Float64}}()
    ifm_normfactors = Dict{String, Float64}()
    ifm_elements = JulES.TuLiPa.DataElement[]
    
    if JulES.get_onlyagghydro(settings)
        detailedrescopl = Dict()
        return Dict("elements" => progelements, "detailedrescopl" => detailedrescopl,
            "iprogtype" => iprogtype, "ifm_replacemap" => ifm_replacemap, "ifm_names" => ifm_names, "ifm_weights" => ifm_weights, "ifm_normfactors" => ifm_normfactors, "ifm_elements" => ifm_elements)
    else
        elements = vcat(themaseries, themastructure_exl, detdseries, detdstructure)
        detailedrescopl = JSON.parsefile(joinpath(sti_vannkraft, "magasin_elspot.json"))
        return Dict("elements" => elements, "elements_ppp" => progelements, "detailedrescopl" => detailedrescopl,
            "iprogtype" => iprogtype, "ifm_replacemap" => ifm_replacemap, "ifm_names" => ifm_names, "ifm_weights" => ifm_weights, "ifm_normfactors" => ifm_normfactors, "ifm_elements" => ifm_elements)
    end
end

### Run JulES and keep the results

In [ ]:
dataset = getdataset(config, weatheryear)
input = JulES.DefaultJulESInput(config, dataset, datayear, weatheryear)
JulES.cleanup_jules(input)
data = JulES.run_serial(input)

### Show some of the results

In [ ]:
powerbalancenames = data["areanames"]
prices = data["pricematrix"]
x1 = data["priceindex"]

hydronames = data["resnames"]
hydrolevels = data["resmatrix"]
x2 = data["resindex"]

batterynames = data["batnames"]
batterylevels = data["batmatrix"]
x2 = data["batindex"]

statenames = data["statenames"]
statematrix = data["statematrix"]
x3 = data["stateindex"]

supplyvalues = data["supplyvalues"]
supplynames = data["supplynames"]
supplybalancenames = data["supplybalancenames"]

demandvalues = data["demandvalues"]
demandnames = data["demandnames"]
demandbalancenames = data["demandbalancenames"];

# storagevalues = data["storagevalues"]
# storagenames = data["storagenames"]
# scenarionames = data["scenarionames"]
# shorts = data["shorts"]
# skipfactor = data["skipfactor"];

In [ ]:
# Plot prices
idxwohub = findall(x -> !occursin("HUB", x), powerbalancenames) # remove hubs, not active in 2025 dataset
display(plot(x1, prices[:,idxwohub]*100, labels=reshape(powerbalancenames[idxwohub],1,length(powerbalancenames[idxwohub])), size=(800,500), title="Prices", ylabel="€/MWh"))

# # Plot supplies and demands
# maxdemsup = isempty(supplyvalues) ? maximum(demandvalues) : (isempty(demandvalues) ? maximum(supplyvalues) : max(maximum(demandvalues), maximum(supplyvalues)))
# supplychart = plot(x1, supplyvalues,labels=reshape(supplynames,1,length(supplynames)),title="Supply", ylabel = "GWh/h", ylims=(0,maxdemsup))
# demandchart = plot(x1, demandvalues,labels=reshape(demandnames,1,length(demandnames)),title="Demand", ylabel = "GWh/h", ylims=(0,maxdemsup))
sumsupplyvalues = sum(supplyvalues,dims=2)
sumdemandvalues = sum(demandvalues,dims=2)
maxdemsup = isempty(sumsupplyvalues) ? maximum(sumdemandvalues) : (isempty(sumdemandvalues) ? maximum(sumsupplyvalues) : max(maximum(sumdemandvalues), maximum(sumsupplyvalues)))
supplychart = areaplot(x1,sumsupplyvalues,title="Supply", ylabel = "GWh/h")
demandchart = areaplot(x1,sumdemandvalues,title="Demand", ylabel = "GWh/h")
display(plot([supplychart,demandchart]...,layout=(1,2),size=(800,500)))

# Plot storages
# display(areaplot(x2, hydrolevels,labels=reshape(hydronames,1,length(hydronames)),size=(800,500),title="Reservoir levels", ylabel = "TWh")) #
display(areaplot(x2, dropdims(sum(hydrolevels,dims=2),dims=2),labels="Total",size=(800,500),title="Reservoir levels", ylabel = "TWh")) #

display(areaplot(x1, dropdims(sum(batterylevels,dims=2),dims=2),labels="Total",size=(800,500),title="Short term storage levels", ylabel = "GWh")) #

# Plot list of yearly mean production and demand for each supply/demand TODO: split demand/supply and transmission
meandemand = dropdims(mean(demandvalues,dims=1),dims=1)
meanproduction = dropdims(mean(supplyvalues,dims=1),dims=1)
supplydf = sort(DataFrame(Supplyname = supplynames, Yearly_supply_TWh = meanproduction*8.76),[:Yearly_supply_TWh], rev = true)
demanddf = sort(DataFrame(Demandname = demandnames, Yearly_demand_TWh = meandemand*8.76),[:Yearly_demand_TWh], rev = true)
supplydf[!,:ID] = collect(1:length(supplynames))
demanddf[!,:ID] = collect(1:length(demandnames))
joineddf = select!(outerjoin(supplydf,demanddf;on=:ID),Not(:ID))
# show(joineddf,allcols=true, allrows=true, nosubheader = true)

# Check that total supply equals total demand
pretty_table(combine(joineddf, [:Yearly_supply_TWh, :Yearly_demand_TWh] .=> sum∘skipmissing), show_subheader = false)

# # Plot list of yearly income and cost for each supply/demand (only works if exogenprices are collected)
# supplyrev = copy(supplyvalues)
# for (i,supplybalancename) in enumerate(supplybalancenames)
#     idx = findfirst(isequal(supplybalancename), powerbalancenames)
#     supplyrev[:,i] .= supplyrev[:,i] .* prices[:,idx]
# end
# demandrev = copy(demandvalues)
# for (i,demandbalancename) in enumerate(demandbalancenames)
#     idx = findfirst(isequal(demandbalancename), powerbalancenames)
#     demandrev[:,i] .= demandrev[:,i] .* prices[:,idx]
# end
# meandemandrev = dropdims(mean(demandrev,dims=1),dims=1)
# meanproductionrev = dropdims(mean(supplyrev,dims=1),dims=1)
# supplyrevdf = sort(DataFrame(Supplyname = supplynames, Yearly_rev_mill€ = meanproductionrev*8.76),[:Yearly_rev_mill€], rev = true)
# demandrevdf = sort(DataFrame(Demandname = demandnames, Yearly_cost_mill€ = meandemandrev*8.76),[:Yearly_cost_mill€], rev = true)
# supplyrevdf[!,:ID] = collect(1:length(supplynames))
# demandrevdf[!,:ID] = collect(1:length(demandnames))
# joinedrevdf = select!(outerjoin(supplyrevdf,demandrevdf;on=:ID),Not(:ID))
# # pretty_table(joinedrevdf,show_subheader = false)

# # Sum revenues and cost
# pretty_table(combine(joinedrevdf, [:Yearly_rev_mill€, :Yearly_cost_mill€] .=> sum∘skipmissing), show_subheader = false)

# # Plot storagevalues for each reservoir and scenarios
# maxlongtermstorages = 10
# maxshorttermstorages = 10
# shortindex = x3
# medindex = x3[1:Int(skipfactor):end]
# numop = length(findall(sn -> occursin("Operative", sn), scenarionames))
# numscen = length(scenarionames) - numop
# j = 0
# k = 0
# for (i, storagename) in enumerate(storagenames)
#     if shorts[i]
#         j += 1
#         j > maxshorttermstorages && continue
#         storagevalues_ = storagevalues[:,:,:]
#         index = shortindex
#     else
#         k += 1
#         k > maxlongtermstorages && continue
#         storagevalues_ = storagevalues[1:Int(skipfactor):end,:,:]
#         index = medindex
#     end
#     p = plot(index, storagevalues_[:,1:numscen,i] * -100, size=(800,500), title="Storagevalues scenario and operative for " * storagename, labels=reshape(scenarionames[1:numscen], 1, numscen), ylabel="€/MWh")
#     plot!(p, index, storagevalues_[:,numscen+1:end,i] * -100, labels=reshape(scenarionames[numscen+1:end], 1, numop), linewidth=5)
#     display(p)
# end